<a href="https://colab.research.google.com/github/mingjian-shi/CSC207Simulator/blob/master/Week1draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import yfinance as yf
import pandas as pd
import numpy as np

In [12]:
def download_prices(tickers, start, end):

    series_dict = {}

    for t in tickers:
        df = yf.download(
            t,
            start=start,
            end=end,
            progress=False,
            auto_adjust=False
        )

        # Flatten MultiIndex columns if necessary
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        # Select price column
        price_col = "Adj Close" if "Adj Close" in df.columns else "Close"

        s = df[price_col]
        if isinstance(s, pd.DataFrame):
            s = s.iloc[:, 0]

        s.name = t
        series_dict[t] = s

    prices = pd.concat(series_dict.values(), axis=1)

    # Clean index
    prices = prices.sort_index()
    prices = prices[~prices.index.duplicated()]

    # Forward fill missing values
    prices = prices.ffill()

    return prices

In [17]:
SECTORS = [
    "XLF", "XLK", "XLE", "XLV", "XLI",
    "XLP", "XLY", "XLB", "XLU", "XLRE"
]

START = "2010-01-01"
END = "2025-01-01"

prices = download_prices(SECTORS, START, END)
prices.head()

,XLF,XLK,XLE,XLV,XLI,XLP,XLY,XLB,XLU,XLRE
Date,,,,,,,,,,
2010-01-04,8.988359,9.390444,17.381092,24.083450,20.981585,17.347876,12.333345,12.069952,8.948222,NaN
2010-01-05,9.153560,9.378339,17.522951,23.847260,21.055666,17.354372,12.378564,12.108981,8.841694,NaN
2010-01-06,9.171915,9.273416,17.732788,24.091066,21.100113,17.341372,12.395013,12.314756,8.893517,NaN
2010-01-07,9.367713,9.237103,17.706196,24.174877,21.329794,17.341372,12.497790,12.218960,8.853209,NaN
2010-01-08,9.312648,9.297626,17.821455,24.212969,21.670595,17.282827,12.493677,12.389257,8.844571,NaN


In [21]:
prices = prices.dropna()
returns = prices.pct_change().dropna()
print(prices.isna().sum())
returns.head()

XLF     0
XLK     0
XLE     0
XLV     0
XLI     0
XLP     0
XLY     0
XLB     0
XLU     0
XLRE    0
dtype: int64


,XLF,XLK,XLE,XLV,XLI,XLP,XLY,XLB,XLU,XLRE
Date,,,,,,,,,,
2015-10-09,-0.006413,0.004358,-0.006485,0.004550,0.003383,0.002229,0.001159,-0.000452,-0.004773,-0.001655
2015-10-12,0.000861,0.001205,-0.012910,0.002630,0.000187,0.002629,0.004760,-0.008607,0.008906,0.006300
2015-10-13,-0.007739,-0.002167,-0.010433,-0.012533,-0.010676,-0.006050,-0.005378,-0.003884,-0.002263,-0.006260
2015-10-14,-0.008233,-0.001930,0.008613,-0.001919,-0.010791,-0.011564,-0.010171,0.008028,-0.000227,0.000000
2015-10-15,0.022717,0.012814,0.017963,0.022328,0.010909,0.009032,0.013008,0.002730,0.014750,-0.002985


In [22]:
print("prices shape:", prices.shape)
print("returns shape:", returns.shape)

prices shape: (2323, 10)
returns shape: (2322, 10)


In [23]:
HORIZON = 21

future_returns = prices.shift(-HORIZON) / prices - 1
future_returns = future_returns.dropna()

print("future_returns shape:", future_returns.shape)
future_returns.head()

future_returns shape: (2302, 10)


,XLF,XLK,XLE,XLV,XLI,XLP,XLY,XLB,XLU,XLRE
Date,,,,,,,,,,
2015-10-08,0.057289,0.073608,0.005909,0.053134,0.031203,-0.003445,0.048821,0.026263,-0.039773,-0.008607
2015-10-09,0.052496,0.058583,0.002466,0.040327,0.019858,-0.012333,0.033196,0.019706,-0.032656,-0.014921
2015-10-12,0.056320,0.050325,0.017781,0.044594,0.019479,-0.012301,0.038033,0.021019,-0.032594,-0.010544
2015-10-13,0.061092,0.052124,0.006682,0.047373,0.031806,-0.007710,0.037466,0.026147,-0.022005,-0.000862
2015-10-14,0.053735,0.044971,-0.024587,0.031199,0.028899,-0.009647,0.038371,-0.002503,-0.031086,-0.006963


In [30]:
def build_features_for_ticker(ticker, price_series, return_series):

    df = pd.DataFrame(index=price_series.index)

    # 1-day return
    df["ret_1d"] = return_series

    # Moving average ratios
    ma10 = price_series.rolling(10).mean()
    ma50 = price_series.rolling(50).mean()
    df["ma_10"] = ma10
    df["ma_50"] = ma50
    df["price_over_ma10"] = price_series / ma10 - 1
    df["price_over_ma50"] = price_series / ma50 - 1

    # Volatility
    df["vol_20"] = return_series.rolling(20).std()

    # =========================
    # RSI (manual implementation)
    # =========================
    delta = price_series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()

    rs = avg_gain / avg_loss
    df["rsi_14"] = 100 - (100 / (1 + rs))

    # =========================
    # MACD (manual)
    # =========================
    ema12 = price_series.ewm(span=12, adjust=False).mean()
    ema26 = price_series.ewm(span=26, adjust=False).mean()

    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()

    df["macd"] = macd
    df["macd_signal"] = signal
    df["macd_diff"] = macd - signal

    # =========================
    # Bollinger Band Width
    # =========================
    rolling_mean = price_series.rolling(20).mean()
    rolling_std = price_series.rolling(20).std()

    upper = rolling_mean + 2 * rolling_std
    lower = rolling_mean - 2 * rolling_std

    df["bb_high"] = upper
    df["bb_low"] = lower
    df["bb_mavg"] = rolling_mean
    df["bb_width"] = (upper - lower) / rolling_mean

    df["ticker"] = ticker

    return df


In [31]:
feature_frames = []

for t in prices.columns:
    df_t = build_features_for_ticker(
        t,
        prices[t],
        returns[t]
    )
    feature_frames.append(df_t)

# Stack vertically
features = pd.concat(feature_frames)

# Drop early NaNs from rolling windows
features = features.dropna()

# Make date a column
features = features.reset_index().rename(columns={"Date": "date"})

In [32]:
target_long = future_returns.stack().reset_index()
target_long.columns = ["date", "ticker", "future_return"]

dataset = features.merge(
    target_long,
    on=["date", "ticker"],
    how="inner"
)

print("Final dataset shape:", dataset.shape)
dataset.head()

Final dataset shape: (22530, 17)


,date,ret_1d,ma_10,ma_50,price_over_ma10,price_over_ma50,vol_20,rsi_14,macd,macd_signal,macd_diff,bb_high,bb_low,bb_mavg,bb_width,ticker,future_return
0,2015-12-17,-0.014321,16.152451,16.163173,-0.000581,-0.001244,0.012842,44.341695,-0.043925,-0.011048,-0.032877,16.827154,15.779300,16.303227,0.064273,XLF,-0.123225
1,2015-12-18,-0.026733,16.063056,16.163923,-0.021885,-0.027989,0.014052,38.756490,-0.077387,-0.024316,-0.053071,16.840634,15.686478,16.263556,0.070966,XLF,-0.103004
2,2015-12-21,0.009871,16.004584,16.169786,-0.008621,-0.018750,0.014294,38.276124,-0.090349,-0.037523,-0.052826,16.818852,15.641747,16.230299,0.072525,XLF,-0.095623
3,2015-12-22,0.009349,15.981721,16.178347,0.002079,-0.010100,0.014503,43.385395,-0.087641,-0.047546,-0.040095,16.793010,15.622611,16.207810,0.072212,XLF,-0.122105
4,2015-12-23,0.011790,15.996501,16.193097,0.012956,0.000658,0.014786,50.428878,-0.069459,-0.051929,-0.017530,16.773905,15.620310,16.197108,0.071222,XLF,-0.117769


In [33]:
# Block 6: Final cleanup + save the Week 1 dataset

# Drop any remaining NaNs to keep Week 2 model training simple
before = len(dataset)
dataset = dataset.dropna()
after = len(dataset)

print("rows before:", before)
print("rows after :", after)
print("dropped   :", before - after)

# Save to a CSV (this is your Week 1 output file)
out_file = "week1_dataset.csv"
dataset.to_csv(out_file, index=False)

print("Saved:", out_file)
print("Columns:", list(dataset.columns))

rows before: 22530
rows after : 22530
dropped   : 0
Saved: week1_dataset.csv
Columns: ['date', 'ret_1d', 'ma_10', 'ma_50', 'price_over_ma10', 'price_over_ma50', 'vol_20', 'rsi_14', 'macd', 'macd_signal', 'macd_diff', 'bb_high', 'bb_low', 'bb_mavg', 'bb_width', 'ticker', 'future_return']


In [ ]:
from google.colab import drive
drive.mount("/content/drive")
dataset.to_csv("/content/drive/MyDrive/week1_dataset.csv", index=False)
print("Saved to Drive: MyDrive/week1_dataset.csv")